In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from plotly import express as px
from tqdm.auto import tqdm
import json
import missingno as msno

# battle

In [3]:
json_path = Path("data/statink-Laplusdestiny").resolve()
with open(json_path) as f:
    data = list()
    for line in tqdm(f):
        data.append(json.loads(line))
        # data.append(line)

0it [00:00, ?it/s]

In [4]:
df = pd.DataFrame(data).dropna(how="all", axis=1)
df["timestamp"] = pd.to_datetime(df["start_at"].apply(lambda x: x["iso8601"])).dt.tz_convert("Asia/Tokyo")
df["rule_name"] = df["rule"].apply(lambda x: x["key"])
df["stage_name"] = df["stage"].apply(lambda x: x["key"])
df["weapon_name"] = df["weapon"].apply(lambda x: x["key"])
df.head()

,id,url,images,user,lobby,rule,stage,weapon,result,knockout,...,start_at,end_at,period,created_at,uuid,bankara_power_after,timestamp,rule_name,stage_name,weapon_name
0,4d02b8c4-5101-4d89-a551-3234109b0eab,https://stat.ink/@Laplusdestiny/spl3/4d02b8c4-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_open', 'name': {'en_US': 'Ana...","{'key': 'asari', 'aliases': ['clam', 'clam_bli...","{'key': 'sumeshi', 'aliases': ['wahoo_world', ...","{'key': 'sputtery', 'aliases': ['dapple_dualie...",win,False,...,"{'time': 1664360534, 'iso8601': '2022-09-28T10...","{'time': 1664360854, 'iso8601': '2022-09-28T10...","{'period': 231161, 'from': '2022-09-28T10:00:0...","{'time': 1664589118, 'iso8601': '2022-10-01T01...",NaN,NaN,2022-09-28 19:22:14+09:00,asari,sumeshi,sputtery
1,6cb80f4b-de9e-403e-b3f3-50a98a9a2bff,https://stat.ink/@Laplusdestiny/spl3/6cb80f4b-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_challenge', 'name': {'en_US':...","{'key': 'yagura', 'aliases': ['loft', 'tower_c...","{'key': 'gonzui', 'aliases': ['alley', 'eeltai...","{'key': 'splatroller', 'aliases': ['splat_roll...",win,False,...,"{'time': 1664361206, 'iso8601': '2022-09-28T10...","{'time': 1664361520, 'iso8601': '2022-09-28T10...","{'period': 231161, 'from': '2022-09-28T10:00:0...","{'time': 1664589123, 'iso8601': '2022-10-01T01...",NaN,NaN,2022-09-28 19:33:26+09:00,yagura,gonzui,splatroller
2,e00005cf-ebb6-4c1e-9a45-c78890ea5cba,https://stat.ink/@Laplusdestiny/spl3/e00005cf-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_challenge', 'name': {'en_US':...","{'key': 'yagura', 'aliases': ['loft', 'tower_c...","{'key': 'gonzui', 'aliases': ['alley', 'eeltai...","{'key': 'splatroller', 'aliases': ['splat_roll...",win,True,...,"{'time': 1664361616, 'iso8601': '2022-09-28T10...","{'time': 1664361765, 'iso8601': '2022-09-28T10...","{'period': 231161, 'from': '2022-09-28T10:00:0...","{'time': 1664589126, 'iso8601': '2022-10-01T01...",NaN,NaN,2022-09-28 19:40:16+09:00,yagura,gonzui,splatroller
3,337a351d-19e6-4675-bc65-3dbfdea97022,https://stat.ink/@Laplusdestiny/spl3/337a351d-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_challenge', 'name': {'en_US':...","{'key': 'yagura', 'aliases': ['loft', 'tower_c...","{'key': 'kinmedai', 'aliases': ['museum', 'mus...","{'key': 'splatroller', 'aliases': ['splat_roll...",lose,True,...,"{'time': 1664361862, 'iso8601': '2022-09-28T10...","{'time': 1664362065, 'iso8601': '2022-09-28T10...","{'period': 231161, 'from': '2022-09-28T10:00:0...","{'time': 1664589130, 'iso8601': '2022-10-01T01...",NaN,NaN,2022-09-28 19:44:22+09:00,yagura,kinmedai,splatroller
4,2036da86-0cf7-4f78-9667-43c0af5c2f3e,https://stat.ink/@Laplusdestiny/spl3/2036da86-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_challenge', 'name': {'en_US':...","{'key': 'yagura', 'aliases': ['loft', 'tower_c...","{'key': 'gonzui', 'aliases': ['alley', 'eeltai...","{'key': 'splatroller', 'aliases': ['splat_roll...",win,True,...,"{'time': 1664362162, 'iso8601': '2022-09-28T10...","{'time': 1664362227, 'iso8601': '2022-09-28T10...","{'period': 231161, 'from': '2022-09-28T10:00:0...","{'time': 1664589134, 'iso8601': '2022-10-01T01...",NaN,NaN,2022-09-28 19:49:22+09:00,yagura,gonzui,splatroller


## medals

In [16]:
medals = list()
medal_all = list()
for i in tqdm(df["medals"]):
    if isinstance(i, list):
        medal_all.extend(i)
    i = pd.Series(i)
    medals.append(i)
medals = pd.DataFrame(medals).rename(columns={0: "medal1", 1: "medal2", 2: "medal3"})
medals

  0%|          | 0/2107 [00:00<?, ?it/s]

,medal1,medal2,medal3
0,バトル No.1,塗りポイント No.1,アシスト数 No.1
1,トドメ数 No.1,カウント進行 No.1,ベストカウント更新
2,バトル No.1,カウント進行 No.1,ベストカウント更新
3,ベストカウント更新,味方のジャンプ先 No.1,グレートバリア No.1
4,移動距離 No.1,ベストカウント更新,ガチヤグラブロック No.1
...,...,...,...
2102,デコイチラシ No.1,注目された時間 No.2,NaN
2103,ナイスダマ No.1,移動距離 No.1,アシスト数 No.1
2104,ナイスダマ No.1,味方のジャンプ先 No.1,ベストカウント更新
2105,アシスト数 No.1,味方のジャンプ先 No.1,ベストカウント更新


In [18]:
medal_all = pd.Series(medal_all)
medal_vc = medal_all.value_counts().reset_index().rename(columns={"index": "medal"})
medal_vc["percent"] = (medal_vc["count"] / medal_vc["count"].sum()).apply(lambda x: f"{x * 100: .2f}%")
medal_vc

,medal,count,percent
0,バトル No.1,510,8.58%
1,味方のジャンプ先 No.1,464,7.80%
2,移動距離 No.1,440,7.40%
3,アシスト数 No.1,423,7.12%
4,注目された時間 No.1,403,6.78%
5,トドメ数 No.1,397,6.68%
6,ベストカウント更新,283,4.76%
7,塗りポイント No.1,282,4.74%
8,カウント進行 No.1,275,4.63%
9,ファーストスプラット,160,2.69%


In [20]:
px.bar(
    medal_vc.query("count >= 100"),
    x="medal",
    y="count",
    text="percent",
    height=600,
    # width=800,
    title="表彰獲得回数とその割合"
)

## battle counts

In [21]:
df = df.merge(medals, left_index=True, right_index=True)
df.head()

,id,url,images,user,lobby,rule,stage,weapon,result,knockout,...,timestamp,rule_name,stage_name,weapon_name,medal1_x,medal2_x,medal3_x,medal1_y,medal2_y,medal3_y
0,4d02b8c4-5101-4d89-a551-3234109b0eab,https://stat.ink/@Laplusdestiny/spl3/4d02b8c4-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_open', 'name': {'en_US': 'Ana...","{'key': 'asari', 'aliases': ['clam', 'clam_bli...","{'key': 'sumeshi', 'aliases': ['wahoo_world', ...","{'key': 'sputtery', 'aliases': ['dapple_dualie...",win,False,...,2022-09-28 19:22:14+09:00,asari,sumeshi,sputtery,バトル No.1,塗りポイント No.1,アシスト数 No.1,バトル No.1,塗りポイント No.1,アシスト数 No.1
1,6cb80f4b-de9e-403e-b3f3-50a98a9a2bff,https://stat.ink/@Laplusdestiny/spl3/6cb80f4b-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_challenge', 'name': {'en_US':...","{'key': 'yagura', 'aliases': ['loft', 'tower_c...","{'key': 'gonzui', 'aliases': ['alley', 'eeltai...","{'key': 'splatroller', 'aliases': ['splat_roll...",win,False,...,2022-09-28 19:33:26+09:00,yagura,gonzui,splatroller,トドメ数 No.1,カウント進行 No.1,ベストカウント更新,トドメ数 No.1,カウント進行 No.1,ベストカウント更新
2,e00005cf-ebb6-4c1e-9a45-c78890ea5cba,https://stat.ink/@Laplusdestiny/spl3/e00005cf-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_challenge', 'name': {'en_US':...","{'key': 'yagura', 'aliases': ['loft', 'tower_c...","{'key': 'gonzui', 'aliases': ['alley', 'eeltai...","{'key': 'splatroller', 'aliases': ['splat_roll...",win,True,...,2022-09-28 19:40:16+09:00,yagura,gonzui,splatroller,バトル No.1,カウント進行 No.1,ベストカウント更新,バトル No.1,カウント進行 No.1,ベストカウント更新
3,337a351d-19e6-4675-bc65-3dbfdea97022,https://stat.ink/@Laplusdestiny/spl3/337a351d-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_challenge', 'name': {'en_US':...","{'key': 'yagura', 'aliases': ['loft', 'tower_c...","{'key': 'kinmedai', 'aliases': ['museum', 'mus...","{'key': 'splatroller', 'aliases': ['splat_roll...",lose,True,...,2022-09-28 19:44:22+09:00,yagura,kinmedai,splatroller,ベストカウント更新,味方のジャンプ先 No.1,グレートバリア No.1,ベストカウント更新,味方のジャンプ先 No.1,グレートバリア No.1
4,2036da86-0cf7-4f78-9667-43c0af5c2f3e,https://stat.ink/@Laplusdestiny/spl3/2036da86-...,"{'judge': None, 'results': None, 'gear': None}","{'id': 13408, 'name': 'らぷらす', 'screen_name': '...","{'key': 'bankara_challenge', 'name': {'en_US':...","{'key': 'yagura', 'aliases': ['loft', 'tower_c...","{'key': 'gonzui', 'aliases': ['alley', 'eeltai...","{'key': 'splatroller', 'aliases': ['splat_roll...",win,True,...,2022-09-28 19:49:22+09:00,yagura,gonzui,splatroller,移動距離 No.1,ベストカウント更新,ガチヤグラブロック No.1,移動距離 No.1,ベストカウント更新,ガチヤグラブロック No.1


In [23]:
vc = df.value_counts(subset=["rule_name", "stage_name", "weapon_name"]).reset_index(name="count")
vc

,rule_name,stage_name,weapon_name,count
0,yagura,hirame,clashblaster,31
1,yagura,hirame,longblaster,31
2,hoko,yunohana,variableroller,30
3,hoko,amabi,variableroller,26
4,hoko,mahimahi,variableroller,25
...,...,...,...,...
521,hoko,amabi,sputtery,1
522,nawabari,yunohana,splatroller,1
523,nawabari,yunohana,splatroller_collabo,1
524,hoko,amabi,sharp,1


In [24]:
px.histogram(
    vc, 
    x="stage_name", 
    y="count", 
    color="weapon_name", 
    facet_col="rule_name", 
    height=800, 
    # barmode="overlay"
)

# salmon

In [25]:
salmon_path = Path("data/statink-salmon-Laplusdestiny").resolve()
with open(salmon_path) as f:
    salmon = list()
    for line in tqdm(f):
        salmon.append(json.loads(line))
        # data.append(line)

0it [00:00, ?it/s]

In [26]:
salmon_df = pd.DataFrame(salmon).dropna(how="all", axis=1)
salmon_df["timestamp"] = pd.to_datetime(salmon_df["start_at"].apply(lambda x: x["iso8601"])).dt.tz_convert("Asia/Tokyo")
# salmon_df["rule_name"] = salmon_df["rule"].apply(lambda x: x["key"])
salmon_df["stage_name"] = salmon_df["stage"].apply(lambda x: x["key"])
# salmon_df["weapon_name"] = salmon_df["weapon"].apply(lambda x: x["key"])
salmon_df.head()

,id,url,user,uuid,private,big_run,stage,danger_rate,clear_waves,fail_reason,...,game_version,user_agent,automated,start_at,period,shift,created_at,eggstra_work,timestamp,stage_name
0,91d07037-a33d-48be-81b0-2e03552c49b3,https://stat.ink/@Laplusdestiny/salmon3/91d070...,"{'id': 13408, 'name': 'らぷらす', 'screen_name': '...",61c1a7ac-41c0-5ee1-b439-4e3e4e630478,False,False,"{'key': 'meuniere', 'aliases': ['7', 'gone_fis...",178.4,2.0,"{'key': 'time_limit', 'name': {'en_US': 'Time ...",...,"{'tag': '1.2.1', 'name': {'en_US': 'v1.2.1', '...","{'name': 's3s', 'version': 'v0.2.0', 'variable...",True,"{'time': 1668003197, 'iso8601': '2022-11-09T14...","{'period': 231667, 'from': '2022-11-09T14:00:0...","{'from': '2022-11-09T00:00:00+00:00', 'to': '2...","{'time': 1669450448, 'iso8601': '2022-11-26T08...",NaN,2022-11-09 23:13:17+09:00,meuniere
1,cf0054c9-3c55-49e3-a4f0-ac7e99aa96a8,https://stat.ink/@Laplusdestiny/salmon3/cf0054...,"{'id': 13408, 'name': 'らぷらす', 'screen_name': '...",822d9e7c-26f1-5571-884a-1578bf784370,False,False,"{'key': 'meuniere', 'aliases': ['7', 'gone_fis...",178.4,2.0,"{'key': 'time_limit', 'name': {'en_US': 'Time ...",...,"{'tag': '1.2.1', 'name': {'en_US': 'v1.2.1', '...","{'name': 's3s', 'version': 'v0.2.0', 'variable...",True,"{'time': 1668003605, 'iso8601': '2022-11-09T14...","{'period': 231667, 'from': '2022-11-09T14:00:0...","{'from': '2022-11-09T00:00:00+00:00', 'to': '2...","{'time': 1669450450, 'iso8601': '2022-11-26T08...",NaN,2022-11-09 23:20:05+09:00,meuniere
2,f119928d-75f2-4eb1-bbfd-82d9a4f88623,https://stat.ink/@Laplusdestiny/salmon3/f11992...,"{'id': 13408, 'name': 'らぷらす', 'screen_name': '...",1882dfce-6b88-56f8-9eb6-63749e433ad4,False,False,"{'key': 'meuniere', 'aliases': ['7', 'gone_fis...",175.4,0.0,"{'key': 'time_limit', 'name': {'en_US': 'Time ...",...,"{'tag': '1.2.1', 'name': {'en_US': 'v1.2.1', '...","{'name': 's3s', 'version': 'v0.2.0', 'variable...",True,"{'time': 1668004021, 'iso8601': '2022-11-09T14...","{'period': 231667, 'from': '2022-11-09T14:00:0...","{'from': '2022-11-09T00:00:00+00:00', 'to': '2...","{'time': 1669450453, 'iso8601': '2022-11-26T08...",NaN,2022-11-09 23:27:01+09:00,meuniere
3,a2c1f9bf-b892-49b4-9213-d921e83d607b,https://stat.ink/@Laplusdestiny/salmon3/a2c1f9...,"{'id': 13408, 'name': 'らぷらす', 'screen_name': '...",a3667eb8-2414-5adb-bd28-af44d345921a,False,False,"{'key': 'meuniere', 'aliases': ['7', 'gone_fis...",172.4,1.0,"{'key': 'time_limit', 'name': {'en_US': 'Time ...",...,"{'tag': '1.2.1', 'name': {'en_US': 'v1.2.1', '...","{'name': 's3s', 'version': 'v0.2.0', 'variable...",True,"{'time': 1668004207, 'iso8601': '2022-11-09T14...","{'period': 231667, 'from': '2022-11-09T14:00:0...","{'from': '2022-11-09T00:00:00+00:00', 'to': '2...","{'time': 1669450456, 'iso8601': '2022-11-26T08...",NaN,2022-11-09 23:30:07+09:00,meuniere
4,72f96071-1dac-4357-af62-2affb1de56fd,https://stat.ink/@Laplusdestiny/salmon3/72f960...,"{'id': 13408, 'name': 'らぷらす', 'screen_name': '...",5a5f4c44-c453-5d05-8cbc-9b8ee85d7477,False,False,"{'key': 'meuniere', 'aliases': ['7', 'gone_fis...",170.4,0.0,"{'key': 'time_limit', 'name': {'en_US': 'Time ...",...,"{'tag': '1.2.1', 'name': {'en_US': 'v1.2.1', '...","{'name': 's3s', 'version': 'v0.2.0', 'variable...",True,"{'time': 1668004495, 'iso8601': '2022-11-09T14...","{'period': 231667, 'from': '2022-11-09T14:00:0...","{'from': '2022-11-09T00:00:00+00:00', 'to': '2...","{'time': 1669450458, 'iso8601': '2022-11-26T08...",NaN,2022-11-09 23:34:55+09:00,meuniere


## キケン度

In [40]:
px.histogram(
    salmon_df.query("danger_rate >= 50").sort_values(by="clear_waves"), 
    x="danger_rate", 
    height=800,
    color="clear_waves",
    title="キケン度",
    barmode="overlay",
    nbins=20,
    marginal="box",
    # histnorm="probability"
)

## ボス討伐数

In [53]:
bosslist = list()
for i in tqdm(salmon_df["bosses"]):
    # display(pd.DataFrame(i).T)
    bosslist.append(pd.DataFrame(i).T.reset_index())
bosslist = pd.concat(bosslist)
bosslist = bosslist.drop(columns="boss").rename(columns={"index": "boss"})
bosslist.head()
# tmp = pd.DataFrame(salmon_df.loc[0]["bosses"]).T
# tmp

  0%|          | 0/325 [00:00<?, ?it/s]

,boss,appearances,defeated,defeated_by_me
0,bakudan,6,6,2
1,diver,3,3,1
2,hashira,5,2,0
3,hebi,5,5,2
4,katapad,3,3,0


In [61]:
boss = bosslist.groupby("boss").sum().reset_index()
boss["percent"] = boss["defeated_by_me"] / boss["appearances"]
boss.sort_values(by="percent", ascending=False)

,boss,appearances,defeated,defeated_by_me,percent
9,mogura,1342,1243,384,0.28614
1,diver,1274,1128,338,0.265306
4,hashira,1253,955,275,0.219473
12,teppan,1402,1281,294,0.2097
7,kin_shake,449,399,94,0.209354
5,hebi,1392,1332,290,0.208333
0,bakudan,1374,1212,284,0.206696
11,tekkyu,532,419,93,0.174812
2,doro_shake,87,70,13,0.149425
8,koumori,1381,889,187,0.135409


In [65]:
px.bar(
    boss.sort_values(by="percent", ascending=False), 
    x="boss", 
    y="percent",
    title="ボスの倒した割合"
)